In [1]:
import transformers
import PIL
import torch


CLIP interrogator

In [2]:
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import torch

# Load the pre-trained CLIP model and processor
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Load the image
image = Image.open("barbara.jpg")

# Define text prompts for both male and female models, including additional descriptors
inputs = processor(
    text=[
        "a woman", "a female model", "a man", "a male model", "long hair", "short hair", 
        "blonde hair", "brunette hair", "black hair", "red hair", "blue eyes", "brown eyes", 
        "green eyes", "wearing a dress", "wearing pants", "wearing a suit", "formal clothing", 
        "casual clothing", "elegant outfit", "athletic wear", "shirtless", "wearing accessories", 
        "wearing a hat", "wearing sunglasses", "wearing jewelry", "smiling", "posing", "a confident pose", 
        "a shy expression", "fashionable", "glamorous", "professional look", "muscular build", "slim build", 
        "red color", "white color", "black color", "colorful outfit", "a person", "a weapon", "clothing", 
        "an accessory", "a power", "underage"
    ], 
    images=image, 
    return_tensors="pt", 
    padding=True
)

# Get the model's outputs
outputs = model(**inputs)

# Extract image-text similarity logits
image_logits = outputs.logits_per_image
text_logits = outputs.logits_per_text

# Convert logits to probabilities using softmax
probs = image_logits.softmax(dim=1).cpu().detach().numpy()

# Define labels corresponding to the prompts used above
labels = [
    "a woman", "a female model", "a man", "a male model", "long hair", "short hair", 
    "blonde hair", "brunette hair", "black hair", "red hair", "blue eyes", "brown eyes", 
    "green eyes", "wearing a dress", "wearing pants", "wearing a suit", "formal clothing", 
    "casual clothing", "elegant outfit", "athletic wear", "shirtless", "wearing accessories", 
    "wearing a hat", "wearing sunglasses", "wearing jewelry", "smiling", "posing", "a confident pose", 
    "a shy expression", "fashionable", "glamorous", "professional look", "muscular build", "slim build", 
    "red color", "white color", "black color", "colorful outfit", "a person", "a weapon", "clothing", 
    "an accessory", "a power", "underage"
]

# Map the probabilities to their respective labels
label_probs = {label: prob for label, prob in zip(labels, probs[0])}

# Print the probabilities associated with each label
print(label_probs)


c:\Users\sharo\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\sharo\anaconda3\Lib\site-packages\transformers\modeling_utils.py:463: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.ser

{'a woman': 0.016404908, 'a female model': 0.39563328, 'a man': 0.00062206626, 'a male model': 0.023173759, 'long hair': 0.0039416337, 'short hair': 0.00056477747, 'blonde hair': 0.0040155826, 'brunette hair': 0.00088862143, 'black hair': 8.172985e-06, 'red hair': 0.0003158762, 'blue eyes': 0.00078156404, 'brown eyes': 6.432954e-05, 'green eyes': 0.00059454853, 'wearing a dress': 0.0016326737, 'wearing pants': 0.0027476654, 'wearing a suit': 0.23180756, 'formal clothing': 0.0520163, 'casual clothing': 0.0035890695, 'elegant outfit': 0.016313393, 'athletic wear': 0.0014441503, 'shirtless': 4.055542e-05, 'wearing accessories': 0.0058673625, 'wearing a hat': 0.0007629613, 'wearing sunglasses': 0.00077480765, 'wearing jewelry': 0.009481276, 'smiling': 0.0005229917, 'posing': 0.0033029204, 'a confident pose': 0.0012674065, 'a shy expression': 0.0006933988, 'fashionable': 0.026015729, 'glamorous': 0.04000486, 'professional look': 0.0144398585, 'muscular build': 1.206879e-05, 'slim build': 0.

Deepdanbooru AUTOMATIC1111 implementation for image labeling

In [17]:
import base64
import io
from PIL import Image

# def encode(f):
#     image = Image.open(f)
#     exif = image.getexif()
#     if image.mode == 'RGBA':
#         image = image.convert('RGB')
#     with io.BytesIO() as stream:
#         image.save(stream, 'JPEG', exif = exif)
#         values = stream.getvalue()
#         encoded = base64.b64encode(values).decode()
#         return encoded
def encode(f):
    image = Image.open(f)
    exif = image.getexif()
    
    # Convert image to RGB if it's not already
    if image.mode != 'RGB':
        image = image.convert('RGB')
    
    with io.BytesIO() as stream:
        image.save(stream, format='JPEG', exif=exif)
        values = stream.getvalue()
        encoded = base64.b64encode(values).decode()
        return encoded



def decode(encoding):
    if encoding.startswith("data:image/"):
        encoding = encoding.split(";")[1].split(",")[1]
    return Image.open(io.BytesIO(base64.b64decode(encoding)))

In [18]:
import requests

image_path = "barbara.jpg"

def interrogate_sd(image_path, model="deepdanbooru"):
    payload = {
        'image': encode(image_path),
        "model": model,
    }

    response = requests.post(url=f'http://127.0.0.1:7860/sdapi/v1/interrogate', json=payload)
    # response = requests.post(url='http://127.0.0.1:7860/sdapi/v1/interrogate', json=payload)
    if(response.status_code != 200):
        print(f"Request failed with status code {response.status_code}")
        return
    return response.json()['caption']

In [19]:
import os

label_folder = 'character_images'

def get_file_paths(root_folder):
    file_paths = []
    for root, dirs, files in os.walk(root_folder):
        for file in files:
            file_path = os.path.join(root, file)
            file_paths.append(file_path)
    return file_paths

all_file_paths = get_file_paths(label_folder)

#Print all file paths
for path in all_file_paths:
    print(path)
    

character_images\Aaron_Taylor-Johnson\image_000.jpg
character_images\Aaron_Taylor-Johnson\image_001.jpg
character_images\Aaron_Taylor-Johnson\image_002.jpg
character_images\Aaron_Taylor-Johnson\image_003.jpg
character_images\Aaron_Taylor-Johnson\image_004.jpg
character_images\Aaron_Taylor-Johnson\image_005.jpg
character_images\Aaron_Taylor-Johnson\image_006.jpg
character_images\Aaron_Taylor-Johnson\image_007.jpg
character_images\Aaron_Taylor-Johnson\image_008.jpg
character_images\Aaron_Taylor-Johnson\image_009.jpg
character_images\Aaron_Taylor-Johnson\image_010.jpg
character_images\Aaron_Taylor-Johnson\image_011.jpg
character_images\Aaron_Taylor-Johnson\image_012.jpg
character_images\Aaron_Taylor-Johnson\image_013.jpg
character_images\Aaron_Taylor-Johnson\image_014.jpg
character_images\Aaron_Taylor-Johnson\image_015.jpg
character_images\Aaron_Taylor-Johnson\image_016.jpg
character_images\Aaron_Taylor-Johnson\image_017.jpg
character_images\Aaron_Taylor-Johnson\image_018.jpg
character_im

In [29]:

image_labels = {}
for file in all_file_paths:
    name = file.split("\\")[-1]
    image_labels[name] = interrogate_sd(file)

print(image_labels)
	

Response status: 404
Response content: {"detail":"Not Found"}
Response status: 404
Response content: {"detail":"Not Found"}
Response status: 404
Response content: {"detail":"Not Found"}
Response status: 404
Response content: {"detail":"Not Found"}
Response status: 404
Response content: {"detail":"Not Found"}
Response status: 404
Response content: {"detail":"Not Found"}
Response status: 404
Response content: {"detail":"Not Found"}
Response status: 404
Response content: {"detail":"Not Found"}
Response status: 404
Response content: {"detail":"Not Found"}
Response status: 404
Response content: {"detail":"Not Found"}
Response status: 404
Response content: {"detail":"Not Found"}
Response status: 404
Response content: {"detail":"Not Found"}
Response status: 404
Response content: {"detail":"Not Found"}
Response status: 404
Response content: {"detail":"Not Found"}
Response status: 404
Response content: {"detail":"Not Found"}
Response status: 404
Response content: {"detail":"Not Found"}
Response

KeyboardInterrupt: 

In [ ]:
import os
import torch
from PIL import Image
from transformers import pipeline
import time
import json

def get_file_paths(root_folder):
    file_paths = []
    for root, dirs, files in os.walk(root_folder):
        for file in files:
            file_path = os.path.join(root, file)
            file_paths.append(file_path)
    return file_paths

def caption_image_hf(image_path):
    captioner = pipeline("image-to-text", model="Salesforce/blip-image-captioning-base")
   
    # Open the image using PIL and convert it to RGB mode (if it's not already)
    image = Image.open(image_path).convert("RGB")
   
    # Use the captioner pipeline
    captions = captioner(image)
    return [caption['generated_text'] for caption in captions]

# Specify the folder containing character images
label_folder = 'character_images'

# Get all file paths
all_file_paths = get_file_paths(label_folder)

# Create dictionary to store image labels
image_labels = {}

# Process images with logging and timing
for idx, file in enumerate(all_file_paths):
    start_time = time.time()  # Start time for processing the image
    name = file.split("\\")[-1]
    
    # Log the image being processed
    print(f"Processing image {idx + 1}/{len(all_file_paths)}: {name}")
    
    labels = caption_image_hf(file)
    image_labels[name] = labels
    
    # Log the time taken for processing
    end_time = time.time()
    print(f"Time taken for {name}: {end_time - start_time:.2f} seconds")

# Print image labels after processing
print(image_labels)

# Save image labels to a JSON file
with open('image_labels.json', 'w') as f:
    json.dump(image_labels, f, indent=4)

print("Image labels saved to image_labels.json")

# Save image labels to a text file
with open('image_labels.txt', 'w') as f:
    for image_name, labels in image_labels.items():
        f.write(f"{image_name}: {', '.join(labels)}\n")

print("Image labels saved to image_labels.txt")


Processing image 1/9999: image_000.jpg
Time taken for image_000.jpg: 16.99 seconds
Processing image 2/9999: image_001.jpg
Time taken for image_001.jpg: 5.60 seconds
Processing image 3/9999: image_002.jpg
Time taken for image_002.jpg: 7.30 seconds
Processing image 4/9999: image_003.jpg
Time taken for image_003.jpg: 7.16 seconds
Processing image 5/9999: image_004.jpg
Time taken for image_004.jpg: 5.84 seconds
Processing image 6/9999: image_005.jpg
Time taken for image_005.jpg: 6.80 seconds
Processing image 7/9999: image_006.jpg
Time taken for image_006.jpg: 7.07 seconds
Processing image 8/9999: image_007.jpg
Time taken for image_007.jpg: 8.38 seconds
Processing image 9/9999: image_008.jpg
Time taken for image_008.jpg: 15.26 seconds
Processing image 10/9999: image_009.jpg
Time taken for image_009.jpg: 7.74 seconds
Processing image 11/9999: image_010.jpg
Time taken for image_010.jpg: 6.86 seconds
Processing image 12/9999: image_011.jpg
Time taken for image_011.jpg: 6.42 seconds
Processing 

In [42]:
import json

with open('character-labeling.json', 'w') as fp:
    json.dump(image_lables, fp)